In [3]:
!pip install matplotlib
!pip install seaborn 
!pip install numpy
!pip install rdata
!pip install pyreadr

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path
import os
import pickle
import rdata
import pyreadr

In [30]:
#Read in all data
data = pd.read_csv("clim_conflict_for_R.csv")
area = pd.read_csv("croparea_weight.csv", encoding="windows-1252")
ssa = pd.read_csv("gseasSSA.csv")
precip = pd.read_csv("precip_climatology.csv", index_col=0)
mss_country = pd.read_csv("mss_countries_new.csv", header=0)
mss_country.iloc[:, 0] = mss_country.iloc[:, 0].astype(str)
mss = mss_country.iloc[:, 0].unique().tolist()
scenario_df = pd.read_csv("altogether_monthly.csv")

In [52]:
hold = []

# Helper to select columns by index
def select_columns(indices):
    return data.iloc[:, indices].copy()

#Create dataframes for each model 
hold.append(select_columns([105, 41, 44] + list(range(109, 150)) + list(range(177, 218))))  # Model 1
hold.append(select_columns([105, 41, 44, 53, 56] + list(range(109, 150)) + list(range(177, 218))))  # Model 2

hold

[      war_prio_new   temp_all  temp_all_lag  iccode1  iccode2  iccode3  \
 0                0  26.682407     26.681481        1        0        0   
 1                0  26.357407     26.682407        1        0        0   
 2                0  27.250000     26.357407        1        0        0   
 3                0  26.683333     27.250000        1        0        0   
 4                0  26.742593     26.683333        1        0        0   
 ...            ...        ...           ...      ...      ...      ...   
 1044             1  27.927411     27.802211        0        0        0   
 1045             1        NaN           NaN        0        0        0   
 1046             1        NaN           NaN        0        0        0   
 1047             0        NaN           NaN        0        0        0   
 1048             1        NaN           NaN        0        0        0   
 
       iccode4  iccode5  iccode6  iccode7  ...  Iccyear32  Iccyear33  \
 0           0        0   

In [10]:
import pandas as pd
import statsmodels.api as sm
from pathlib import Path

boot_dir = Path(".")  # Save to current working directory (where your notebook is)

for j in range(2):  # Only two models: 0 and 1
    print(f"Running bootstrap for model {j + 1}...")

    # Drop NAs in the second column (adjust if your NA logic is different)
    aa = hold[j].dropna(subset=[hold[j].columns[1]])
    coefs = []

    for i in range(10000):
        sample = aa.sample(n=len(aa), replace=True)
        y = sample.iloc[:, 0]
        X = sm.add_constant(sample.drop(sample.columns[[0]], axis=1))

        try:
            model = sm.OLS(y, X).fit()
            coef_series = model.params.iloc[1:7]  # Adjust range if needed
            coefs.append(coef_series)
        except Exception as e:
            continue

        if i % 1000 == 0:
            print(f"Model {j + 1}, Iteration {i}")

    coefs_df = pd.DataFrame(coefs)
    coefs_df.columns = model.params.index[1:7]
    coefs_df.reset_index(drop=True, inplace=True)
    coefs_df.to_pickle(boot_dir / f"bootstrap_df_{j + 1}.pkl")
    print(f"Saved bootstrap coefficients DataFrame for model {j + 1}.")


Running bootstrap for model 1...
Model 1, Iteration 0
Model 1, Iteration 1000
Model 1, Iteration 2000
Model 1, Iteration 3000
Model 1, Iteration 4000
Model 1, Iteration 5000
Model 1, Iteration 6000
Model 1, Iteration 7000
Model 1, Iteration 8000
Model 1, Iteration 9000
Saved bootstrap coefficients DataFrame for model 1.
Running bootstrap for model 2...
Model 2, Iteration 0
Model 2, Iteration 1000
Model 2, Iteration 2000
Model 2, Iteration 3000
Model 2, Iteration 4000
Model 2, Iteration 5000
Model 2, Iteration 6000
Model 2, Iteration 7000
Model 2, Iteration 8000
Model 2, Iteration 9000
Saved bootstrap coefficients DataFrame for model 2.


In [51]:
import pandas as pd
import statsmodels.api as sm
from pathlib import Path

boot_dir = Path(".")  # Save to current working directory (where your notebook is)

j = 1  # Only run for model 2 (Python is zero-indexed, so this is the second element in hold)
print(f"Running bootstrap for model {j + 1}...")

# Drop NAs in the second column (adjust if your NA logic is different)
aa = hold[j].dropna(subset=[hold[j].columns[1]])
coefs = []

for i in range(10000):
    sample = aa.sample(n=len(aa), replace=True)
    y = sample.iloc[:, 0]
    X = sm.add_constant(sample.drop(sample.columns[[0]], axis=1))

    try:
        model = sm.OLS(y, X).fit()
        coef_series = model.params.iloc[1:7]  # Adjust range if needed
        coefs.append(coef_series)
    except Exception as e:
        continue

    if i % 1000 == 0:
        print(f"Model {j + 1}, Iteration {i}")

coefs_df = pd.DataFrame(coefs)
coefs_df.columns = model.params.index[1:7]
coefs_df.reset_index(drop=True, inplace=True)
coefs_df.to_pickle(boot_dir / f"bootstrap_df_new_{j + 1}.pkl")
print(f"Saved bootstrap coefficients DataFrame for model {j + 1}.")


Running bootstrap for model 2...
Model 2, Iteration 0
Model 2, Iteration 1000
Model 2, Iteration 2000
Model 2, Iteration 3000
Model 2, Iteration 4000
Model 2, Iteration 5000
Model 2, Iteration 6000
Model 2, Iteration 7000
Model 2, Iteration 8000
Saved bootstrap coefficients DataFrame for model 2.


In [9]:
bootstrap_1df =  pd.read_pickle("bootstrap_df_1.pkl")
bootstrap_1df

,temp_all,temp_all_lag,iccode1,iccode2,iccode3,iccode4
0,0.052061,0.030290,-0.572482,-0.430442,-0.538123,-0.511136
1,0.020082,0.032795,-0.378683,-0.324318,-0.385978,-0.361752
2,0.049877,-0.013532,-0.269753,-0.279535,-0.327668,-0.346560
3,0.049536,-0.002810,-0.304950,-0.347081,-0.390123,-0.374087
4,0.048104,0.019238,-0.453372,-0.373020,-0.461288,-0.437572
...,...,...,...,...,...,...
9995,0.087975,0.016512,-0.493346,-0.557856,-0.646584,-0.638238
9996,0.059384,-0.028289,-0.238343,-0.222097,-0.251246,-0.248003
9997,0.069353,-0.036777,-0.479579,-0.289549,-0.318874,-0.285063
9998,0.007775,-0.021372,-0.129269,-0.122022,-0.100355,-0.105317


In [125]:
bootstrap_df_2 =  pd.read_pickle("bootstrap_df_2.pkl")
bootstrap_df_2

,temp_all,temp_all_lag,prec_all,prec_all_lag,iccode1,iccode2
0,0.051414,0.012759,-0.087864,0.118461,-0.329587,-0.315459
1,0.028335,0.015267,-0.026155,0.050315,-0.444094,-0.305566
2,0.057477,0.006091,-0.049485,0.002407,-0.300446,-0.402271
3,0.035220,0.014401,0.075078,0.026787,-0.389872,-0.303447
4,0.008161,0.041077,-0.015277,0.006991,-0.295048,-0.334502
...,...,...,...,...,...,...
9995,0.044560,-0.011548,-0.015987,0.014224,-0.294776,-0.228590
9996,0.051786,0.009784,0.012148,0.006307,-0.306833,-0.370353
9997,0.028713,0.005569,0.043696,-0.041718,-0.261577,-0.289066
9998,0.043850,0.046556,-0.104647,0.083271,-0.582557,-0.486271


In [21]:
print(scenario_df['Scenario'].unique())

# Split and save by scenario
for scenario in scenario_df['Scenario'].unique():
    scenario_df = scenario_df[scenario_df['Scenario'] == scenario]
    scenario_df.to_csv(f"altogether_monthly_{scenario}.csv", index=False)

['a1b' 'a2' 'b1']


In [22]:
a1b = pd.read_csv("a1b.csv")
a2 = pd.read_csv("a2.csv")
b1 = pd.read_csv("b1.csv")

In [23]:
a1b_clean = a1b.dropna()
a2_clean = a2.dropna()
b1_clean = b1.dropna()

In [ ]:
print(sorted(a1b_clean["Region"].unique()))

In [24]:
african_regions = ['CentrAfrica', 'EastAfrica', 'Sahel', 'SouthAfrica', 'WestAfrica']

a1b_africa = a1b_clean[a1b_clean["Region"].isin(african_regions)]
a2_africa = a2_clean[a2_clean["Region"].isin(african_regions)]
b1_africa = b1_clean[b1_clean["Region"].isin(african_regions)]

In [89]:
median_all = precip.values.flatten()
overall_median = pd.Series(median_all).median()
print(overall_median)

79.3427824853485


In [147]:
precip_median = 5 ## NEED TO CHANGE ##

def bootstrap_median(series, n_bootstrap=10000):
    bootstraps = np.random.choice(series.dropna(), size=(n_bootstrap, len(series)), replace=True)
    medians = np.median(bootstraps, axis=1)
    return np.median(medians)

def table2_stats_temp_precip(df, scenario, precip_median):
    # Model 1: Median absolute temp increase 
    temp_vals = df[df["Variable"] == "temp"]["Value"]
    median_temp1 = bootstrap_median(temp_vals)
    row_model1 = {
        "Scenario": scenario,
        "Model": "Model 1",
        "Median abs temp increase": median_temp1,
        "Median abs precip increase": np.nan
    }

    # Model 2: Median absolute temp increase and abs precip increase 
    temp_vals2 = df[df["Variable"] == "temp"]["Value"]
    median_temp2 = bootstrap_median(temp_vals2)
    precip_vals2 = df[df["Variable"] == "precip"]["Value"]
    median_precip_pct_increase = bootstrap_median(precip_vals2)
    # Convert percent to absolute using the provided precip_median
    median_precip_abs_increase = (median_precip_pct_increase / 100) * precip_median
    row_model2 = {
        "Scenario": scenario,
        "Model": "Model 2",
        "Median abs temp increase": median_temp2,
        "Median abs precip increase": median_precip_abs_increase
    }

    return [row_model1, row_model2]

# Build table using Africa-only DataFrames and your provided precip_median
results = []
for df, scenario in zip([a1b_africa, a2_africa, b1_africa], ["A1B", "A2", "B1"]):
    results.extend(table2_stats_temp_precip(df, scenario, precip_median))

table2_df = pd.DataFrame(results)[[
    "Scenario", "Model", "Median abs temp increase", "Median abs precip increase"
]]
print(table2_df)


  Scenario    Model  Median abs temp increase  Median abs precip increase
0      A1B  Model 1                  1.101874                         NaN
1      A1B  Model 2                  1.101939                    0.086219
2       A2  Model 1                  0.972045                         NaN
3       A2  Model 2                  0.972045                    0.083981
4       B1  Model 1                  0.889395                         NaN
5       B1  Model 2                  0.889308                    0.082221


In [148]:
median_temp_all_model_1 = bootstrap_1df["temp_all"].median() 
median_temp_all_lag_model_1 = bootstrap_1df["temp_all_lag"].median() 
median_temp_all_model_2 = bootstrap_2df["temp_all"].median() 
median_temp_all_lag_model_2 = bootstrap_2df["temp_all_lag"].median()
median_prec_all_model_2 = bootstrap_2df["prec_all"].median()
median_prec_all_lag_model_2 = bootstrap_2df["prec_all_lag"].median()

five_temp_all_model_1 =  bootstrap_1df["temp_all"].quantile(0.05) 
five_temp_all_lag_model_1 = bootstrap_1df["temp_all_lag"].quantile(0.05) 
five_temp_all_model_2 = bootstrap_2df["temp_all"].quantile(0.05) 
five_temp_all_lag_model_2 = bootstrap_2df["temp_all_lag"].quantile(0.05) 
five_prec_all_model_2 = bootstrap_2df["prec_all"].quantile(0.05) 
five_prec_all_lag_model_2 = bootstrap_2df["prec_all_lag"].quantile(0.05) 

ninety_five_temp_all_model_1 =  bootstrap_1df["temp_all"].quantile(0.95) 
ninety_five_temp_all_lag_model_1 = bootstrap_1df["temp_all_lag"].quantile(0.95) 
ninety_five_temp_all_model_2 = bootstrap_2df["temp_all"].quantile(0.95) 
ninety_five_temp_all_lag_model_2 = bootstrap_2df["temp_all_lag"].quantile(0.95) 
ninety_five_prec_all_model_2 = bootstrap_2df["prec_all"].quantile(0.95) 
ninety_five_prec_all_lag_model_2 = bootstrap_2df["prec_all_lag"].quantile(0.95) 

median_temp_model_1 = median_temp_all_model_1 + median_temp_all_lag_model_1
median_temp_model_2 = median_temp_all_model_2 + median_temp_all_lag_model_2
median_prec_model_2 = median_prec_all_model_2 + median_prec_all_lag_model_2
five_temp_model_1 = five_temp_all_model_1 + five_temp_all_lag_model_1
five_temp_model_2 = five_temp_all_model_2 + five_temp_all_lag_model_2
five_prec_model_2 = five_prec_all_model_2 + five_prec_all_lag_model_2
ninety_five_temp_model_1 = ninety_five_temp_all_model_1 + ninety_five_temp_all_lag_model_1
ninety_five_temp_model_2 = ninety_five_temp_all_model_2 + ninety_five_temp_all_lag_model_2
ninety_five_prec_model_2 = ninety_five_prec_all_model_2 + ninety_five_prec_all_lag_model_2


bootstrapped_results = {
    "Variable Name": [
        "median_temp_model_1",
        "median_temp_model_2",
        "median_prec_model_2",
        "five_temp_model_1",
        "five_temp_model_2",
        "five_prec_model_2",
        "ninety_five_temp_model_1",
        "ninety_five_temp_model_2",
        "ninety_five_prec_model_2"
    ],
    "Value": [
        median_temp_model_1,
        median_temp_model_2,
        median_prec_model_2,
        five_temp_model_1,
        five_temp_model_2,
        five_prec_model_2,
        ninety_five_temp_model_1,
        ninety_five_temp_model_2,
        ninety_five_prec_model_2
    ]
}


summary_table = pd.DataFrame(bootstrapped_results)
summary_table

,Variable Name,Value
0,median_temp_model_1,0.055225
1,median_temp_model_2,0.058780
2,median_prec_model_2,0.005240
3,five_temp_model_1,-0.015538
4,five_temp_model_2,-0.018148
5,five_prec_model_2,-0.186199
6,ninety_five_temp_model_1,0.128077
7,ninety_five_temp_model_2,0.138330
8,ninety_five_prec_model_2,0.196475


In [160]:
def calc_median_pct_change(row):
    if row["Model"] == "Model 1":
        # Model 1: median_temp_model_1 * Median abs temp increase
        return median_temp_model_1 * row["Median abs temp increase"] * 100
    else:
        # Model 2: median_temp_model_2 * Median abs temp increase + median_prec_model_2 * Median abs precip increase
        return (
            median_temp_model_2 * row["Median abs temp increase"] * 100 +
            median_prec_model_2 * row["Median abs precip increase"] * 100
        )

def calc_five_pct_change(row):
    if row["Model"] == "Model 1":
        return five_temp_model_1 * row["Median abs temp increase"] * 100
    else:
        # Safe: fillna(0) in case 'Median abs precip increase' is NaN for Model 1
        return (
            five_temp_model_2 * row["Median abs temp increase"] * 100 +
            five_prec_model_2 * (row["Median abs precip increase"] if not pd.isna(row["Median abs precip increase"]) else 0) * 100
        )

def calc_ninety_five_pct_change(row):
    if row["Model"] == "Model 1":
        return ninety_five_temp_model_1 * row["Median abs temp increase"] * 100
    else:
        return (
            ninety_five_temp_model_2 * row["Median abs temp increase"] * 100 +
            ninety_five_prec_model_2 * (row["Median abs precip increase"] if not pd.isna(row["Median abs precip increase"]) else 0) * 100
        )


table_2_final = table2_df
table_2_final["Median % change"] = table_2_final.apply(calc_median_pct_change, axis=1).round(1)

table_2_final= table_2_final.drop("Median abs temp increase", axis = 1)
table_2_final = table_2_final.drop("Median abs precip increase", axis = 1)

table_2_final["% increasein in civil war relative to baseline"] = ((table_2_final["Median % change"] / 11) * 100).round(1)

table_2_final["5th percentile % change"] = table2_df.apply(calc_five_pct_change, axis=1)
table_2_final["95th percentile % change"] = table2_df.apply(calc_ninety_five_pct_change, axis=1)

table_2_final["5th percentile % change"] = table_2_final["5th percentile % change"].round(1)
table_2_final["95th percentile % change"] = table_2_final["95th percentile % change"].round(1)

table_2_final["5th–95th percentile observations of projected % increase"] = (
    table_2_final["5th percentile % change"].astype(str) +
    "–" +
    table_2_final["95th percentile % change"].astype(str)
)

table_2_final = table_2_final.drop(columns=["5th percentile % change", "95th percentile % change"])


cols = list(table_2_final.columns)

civil_war_col = "% increasein in civil war relative to baseline"
percentile_col = "5th–95th percentile observations of projected % increase"

cols.insert(cols.index(civil_war_col), cols.pop(cols.index(percentile_col)))
table_2_final = table_2_final[cols]

def add_proportion_negative_column(table_2_final, bootstrap_1df, bootstrap_2df):
    result = []
    for idx, row in table_2_final.iterrows():
        scenario = row["Scenario"]
        model = row["Model"]
        if model == "Model 1":
            # Use temp_all for Model 1 (same for all scenarios)
            proportion = (bootstrap_1df["temp_all"] < 0).mean() * 100
        else:  # Model 2
            temp_col = f"temp_all_{scenario}"
            precip_col = f"precip_all_{scenario}"
            sum_col = f">0_{scenario}"
            # Compute the columns if they don't exist
            if temp_col not in bootstrap_2df.columns:
                temp_mult = table_2_final.loc[
                    (table_2_final["Scenario"] == scenario) & (table_2_final["Model"] == "Model 2"),
                    "Median abs temp increase"
                ].values[0]
                bootstrap_2df[temp_col] = (bootstrap_2df["temp_all"] + bootstrap_2df["temp_all_lag"]) * temp_mult
            if precip_col not in bootstrap_2df.columns:
                precip_mult = table_2_final.loc[
                    (table_2_final["Scenario"] == scenario) & (table_2_final["Model"] == "Model 2"),
                    "Median abs precip increase"
                ].values[0]
                bootstrap_2df[precip_col] = (bootstrap_2df["prec_all"] + bootstrap_2df["prec_all_lag"]) * precip_mult
            if sum_col not in bootstrap_2df.columns:
                bootstrap_2df[sum_col] = bootstrap_2df[temp_col] + bootstrap_2df[precip_col]
            proportion = (bootstrap_2df[sum_col] < 0).mean() * 100
        result.append(round(proportion, 1))
    table_2_final["% of observations < 0"] = result
    return table_2_final

table_2_final = add_proportion_negative_column(table_2_final, bootstrap_1df, bootstrap_2df)
table_2_final


,Scenario,Model,Median % change,5th–95th percentile observations of projected % increase,% increasein in civil war relative to baseline,% of observations < 0
0,A1B,Model 1,6.1,-1.7–14.1,55.5,1.2
1,A1B,Model 2,6.5,-3.6–16.9,59.1,4.8
2,A2,Model 1,5.4,-1.5–12.4,49.1,1.2
3,A2,Model 2,5.8,-3.3–15.1,52.7,4.9
4,B1,Model 1,4.9,-1.4–11.4,44.5,1.2
5,B1,Model 2,5.3,-3.1–13.9,48.2,5.0


In [163]:
# Add an empty string for Scenario where it is not the first Model in the scenario group
table_2_final['Display Scenario'] = table_2_final['Scenario']
table_2_final.loc[table_2_final.duplicated('Scenario'), 'Display Scenario'] = ""

# Choose and rename columns for presentation
table2_display = table_2_final[[
    'Display Scenario',
    'Model',
    'Median % change',
    '% increasein in civil war relative to baseline',
    '5th–95th percentile observations of projected % increase',
    '% of observations < 0'
]]

# Rename for display to match paper format
table2_display.columns = [
    '',                      # Scenario label
    '',                      # Model label
    'Median % change',
    '% increase in civil war relative to baseline',
    '5th–95th percentile observations of projected % increase',
    '% of observations < 0'
]

print(table2_display.to_string(index=False))


             Median % change  % increase in civil war relative to baseline 5th–95th percentile observations of projected % increase  % of observations < 0
A1B Model 1              6.1                                          55.5                                                -1.7–14.1                    1.2
    Model 2              6.5                                          59.1                                                -3.6–16.9                    4.8
 A2 Model 1              5.4                                          49.1                                                -1.5–12.4                    1.2
    Model 2              5.8                                          52.7                                                -3.3–15.1                    4.9
 B1 Model 1              4.9                                          44.5                                                -1.4–11.4                    1.2
    Model 2              5.3                                          